In [1]:
import pandas as pd
import json
import requests
import ast

In [2]:
df = pd.read_csv("../cleaning/cleaned_products.csv")
LAMBDA_ENDPOINT = "https://hny1katz64.execute-api.us-east-1.amazonaws.com/default/scraped_products"

In [3]:
print(df.columns)
df.head()

Index(['title', 'price', 'info', 'composition', 'usage', 'precautions',
       'brand', 'image', 'description', 'url', 'limpiar', 'tratar', 'proteger',
       'composition_list', 'composition_list_standard', 'id'],
      dtype='object')


,title,price,info,composition,usage,precautions,brand,image,description,url,limpiar,tratar,proteger,composition_list,composition_list_standard,id
0,fotoprotector isdin fusión water pediatrics spf50,s/ 124.90,"['todo tipo de piel, incluso piel atópica y/o ...","vitamina e, provitamina b5, extracto de siempr...",aplica fusion water magic pediatrics spf 50 ge...,uso externo. mantener el producto fuera del al...,isdin,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,fotoprotector isdin fusión water pediatrics sp...,https://inkafarma.pe/producto/fotoprotector-is...,False,False,True,"['vitamina e', 'provitamina b5', 'extracto de ...","['tocoferol', 'extracto de siempreviva', 'vita...",P0000
1,crema anti-irritación la roche posay lipikar 4...,s/ 176.90,"['calma la piel inmediatamente.', 'con su efic...","aqua posae filiformis , microresyl: devuelve e...",aplicar sobre la piel limpia con masaje circul...,mantener fuera del alcance de los niños.,la roche-posay,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,crema anti-irritación la roche posay lipikar b...,https://inkafarma.pe/producto/lipikar-baume-ap...,False,True,False,"['aqua posae filiformis', 'microresyl', 'resta...","['manteca de karité', 'glicerina', 'niacinamid...",P0001
2,serum isdinceutics retinal intense,s/ 299.90,"['ayuda a acelerar la renovación cutánea', 're...","aqua (water), dimethicone, dibutyl adipate, ni...",aplica retinal intense en tu rutina de noche. ...,"si es la primera vez que usas un retinoide, te...",isdin,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,serum isdinceutics retinal intense. el retinal...,https://inkafarma.pe/producto/serum-isdinceuti...,False,True,False,"['aqua (water)', 'dimethicone', 'dibutyl adipa...","['tetrapéptido-7 palmitoil', 'capriloil glicol...",P0002
3,crema en gel hidratante cerave control de grasa,s/ 60.90,"['tecnología de control de grasa', 'no deja la...","agua, niacinamida, glicerina, isononanoato de ...",aplicar generosamente sobre el rostro y el cue...,"uso para adolescentes, jóvenes y adultos. en c...",cerave,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,crema en gel hidratante cerave control de gras...,https://inkafarma.pe/producto/crema-en-gel-hid...,False,True,False,"['agua', 'niacinamida', 'glicerina', 'isononan...","['hialuronato de sodio', 'capriloil glicol', '...",P0003
4,fotoprotector bioderma pediatrics lait,s/ 154.90,"['muy alta protección y multi-resistente.', 'p...","protege, refuerza la delicada piel de los niño...",aplicar sólo sobre piel sana. niños mayores de...,uso especializado para bebés a partir de 12 me...,bioderma,https://dcuk1cxrnzjkh.cloudfront.net/imagespro...,fotoprotector photoderm bioderma pediatrics la...,https://inkafarma.pe/producto/fotoprotector-ph...,False,False,True,"['protege', 'refuerza la delicada piel de los ...",[],P0004


In [4]:
ingredients = ast.literal_eval(df.iloc[0]['composition_list_standard'])
type(ingredients), ingredients

(list, ['tocoferol', 'extracto de siempreviva', 'vitamina b5'])

In [5]:
# Build the payload
products = []
for _, row in df.iterrows():
    try:
        ingredients = ast.literal_eval(row['composition_list_standard'])
        if not isinstance(ingredients, list):
            print("Error, defaulting to empy ingredients")
            ingredients = []
    except:
        print("Error, defaulting to empy ingredients")
        ingredients = []

    # Clean and convert price
    raw_price = str(row['price']).replace('s/', '').replace('$', '').strip()
    try:
        price = float(raw_price)
    except:
        price = None

    product = {
        "name": row['title'],
        "price": price,
        "ingredients": ingredients,
        "description": row['description'],
        "image": None,
        "stars": 0,
        "alt_id": row['id'],
        "brand": row['brand'],
        "limpiar": bool(row.get('limpiar', False)),
        "tratar": bool(row.get('tratar', False)),
        "proteger": bool(row.get('proteger', False))
    }
    products.append(product)


# Send to Lambda
response = requests.post(
    LAMBDA_ENDPOINT,
    headers={"Content-Type": "application/json"},
    data=json.dumps(products)
)

# Print result
print("Status Code:", response.status_code)
try:
    print("Response:", response.json())
except:
    print("Raw Response Text:", response.text)


Status Code: 200
Response: {'message': '148 product(s) inserted successfully'}
